In [ ]:
#Import required python packages
import pandas as pd
import numpy as np
from datetime import datetime
import csv
import os
basepath = '/content/drive/MyDrive/'

#The Belfast Bikes scheme operates 06:00-00:00 daily for pick ups. Allow 2 hours for bike returns and 
#ignore other times to speed up process.
start_time = datetime.strptime('06:00:00',"%H:%M:%S").time()
end_time = datetime.strptime('02:00:00',"%H:%M:%S").time()

#Due to size of dataframe and time that can be run on Google Colab at once need to run through data in
#2 month chunks at a time
start_data = datetime.strptime('2021-06-01 00:00:00', "%Y-%m-%d %H:%M:%S")
end_data = datetime.strptime('2021-08-01 02:00:00', "%Y-%m-%d %H:%M:%S")

#set a minimum journey time of 5 minues and a max journey time of 3 hours. Assume a lot of journeys outside these
#time frames are erroneous entries
min_journey = 300
max_journey = 10800

In [ ]:
#Set up dataframes and read in existing data from earlier run through of this file.

BikesLocationsDF = pd.DataFrame(columns=['BikeID', 'StationID'])
BikesOutDF = pd.DataFrame(columns=['BikeID', 'StationID','DateTimeOut'])
JourneysDF = pd.DataFrame(columns=[ 'BikeID','StationOutID','DateTimeOut','StationInID','DateTimeIn'])

BikesLocationsDF.to_csv(basepath+'BikesLocations.csv', index= False)
BikesOutDF.to_csv(basepath+'BikesOut.csv', index= False)
JourneysDF.to_csv(basepath+'Journeys.csv', index= False)

In [ ]:
#Format columns in DataFrame to required data types
DateParser = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

JourneysDF = pd.read_csv(basepath+'Journeys.csv', dtype={"BikeID": int, "StationOutID": int,
                                                             "StationInID": int},                                                           
                                                            parse_dates=['DateTimeOut','DateTimeIn'],
                                                            date_parser=DateParser)

BikesOutDF = pd.read_csv(basepath+'BikesOut.csv', dtype={"BikeID": int, "StationID": int},
                                                            parse_dates=['DateTimeOut'],
                                                            date_parser=DateParser)

BikesLocationsDF = pd.read_csv(basepath+'BikesLocations.csv', dtype={"BikeID": int, "StationID": int})   

StationsDF = pd.read_csv(basepath+'CurrentStations.csv', dtype={"StationID": int, "StationName": str,
                                                                "Latitude":float, "Longitude": float})



In [ ]:
#Read in main journey dataframe, then take subset based on the start and end dates set up earlier 

APIData= pd.read_csv(basepath+'AllDataFINAL.csv')

APIData['DateTime'] = pd.to_datetime(APIData['DateTime'])

APISubset= APIData.loc[(APIData['DateTime'] > start_data)&(APIData['DateTime'] < end_data)]

In [ ]:

#Set up a list of unique date time values where an event occured.
DateTimeList = pd.DataFrame(columns=['DateTime'])
DateTimeList['DateTime']= APISubset['DateTime'].drop_duplicates()
DateTimeList.sort_values(by="DateTime",ascending=True)



In [ ]:
#Merge the journey dataframe with the stations to get station level information

APISubset = pd.merge(APISubset,StationsDF[['StationName','StationID']],on='StationName', how='left')


In [ ]:
#Set up the "current" dataframe locations of bikes in stations and bikes out on road.

CurrentBikesLocationsDF = pd.DataFrame(columns=['BikeID', 'StationID'])
CurrentBikesOutDF = pd.DataFrame(columns=['BikeID', 'StationID','DateTimeOut'])
CurrentBikesOutDF['BikeID'] = pd.to_numeric(CurrentBikesOutDF['BikeID'])
CurrentBikesOutDF['StationID'] = pd.to_numeric(CurrentBikesOutDF['StationID'])
CurrentBikesOutDF['DateTimeOut'] = pd.to_datetime(CurrentBikesOutDF['DateTimeOut'])   


#iterate through all the times when an event occured

for times in DateTimeList['DateTime']:

  #Format the time in this iteration
  CurrentTime = datetime.strptime(times.strftime("%H:%M:%S"), '%H:%M:%S').time()
  CurrentDateTime = times

  #If the time is within the operational hours of the Belfast Bike Scheme....
  if ((CurrentTime> start_time) or (CurrentTime< end_time)):
 
    #Move all "current" bike data to "previous" dataframes in readiness to check with next time period
    PreviousBikesLocationsDF = CurrentBikesLocationsDF
    PreviousBikesOutDF = CurrentBikesOutDF
    CurrentBikesLocationsDF = pd.DataFrame(columns=['BikeID', 'StationID'])
    

    #Set the current bike locations 

    #Iterate through all rows, and split bikes in each row. (Each row can contain multiple bikes, store all bikes in holdlist)
    for index, row in APISubset.loc[(APISubset['DateTime'] == times)].iterrows():
      holdlist = [i.strip('"[] ') for i in row.BikeAvailable.split(',')]

    #for each bike, if there exists a bike in the row, find the stationID associated with it at this time
    #that is if it exists there it must be in station and can be added to the CurrentBikesLocationsDF
      for bike in holdlist:
        if bike:      
          BikeInStation=[bike,row.loc['StationID']]
          CurrentBikesLocationsDF = CurrentBikesLocationsDF.append(pd.DataFrame([BikeInStation],columns=['BikeID', 'StationID']),
                                                                  ignore_index = True)
                                                                                                    
    CurrentBikesLocationsDF = CurrentBikesLocationsDF.astype( dtype={'BikeID' :int, 'StationID': int})
   



    #Find Bikes that are still in a station now and also previous period - these bikes have not moved
    
    is_duplicate_in_current = CurrentBikesLocationsDF['BikeID'].isin(PreviousBikesLocationsDF['BikeID'])
    is_duplicate_in_previous = PreviousBikesLocationsDF['BikeID'].isin(CurrentBikesLocationsDF['BikeID'])

    #Bikes that are now in a station, but weren't in previous period are added to returns dataframe
    ReturnsDF = CurrentBikesLocationsDF[~is_duplicate_in_current].copy()
    ReturnsDF = ReturnsDF.reset_index(drop=True) 
    ReturnsDF['DateTimeIn']=CurrentDateTime
    ReturnsDF=ReturnsDF.rename(columns={'StationID': 'StationInID','BikeID': 'BikeInID'})
    
    #Create DF of potential journeys to add to the master journey list (only potential as journey may be deemed
    # too long or too short)
    JourneysToAdd = ReturnsDF.merge(PreviousBikesOutDF, left_on=['BikeInID'], 
                                              right_on=['BikeID'], how='inner')
    JourneysToAdd.drop('BikeID', axis=1, inplace=True)
    JourneysToAdd=JourneysToAdd.rename(columns={'StationID': 'StationOutID', 'BikeInID':'BikeID'})

    #format date then find journey length  
    JourneysToAdd['DateTimeOut']=(pd.to_datetime(JourneysToAdd['DateTimeOut'])).dt.strftime("%m/%d/%Y, %H:%M:%S")
    JourneysToAdd['DateTimeOut']=pd.to_datetime(JourneysToAdd['DateTimeOut'])
    JourneysToAdd['JourneyTime']=(JourneysToAdd['DateTimeIn']-JourneysToAdd['DateTimeOut']).dt.total_seconds().astype(int)
    JourneysToAdd = JourneysToAdd.reindex(columns=['BikeID','StationOutID','DateTimeOut','StationInID','DateTimeIn','JourneyTime'])
    

    #Updating Bikes Out Data by removing bikes that have returned
    CurrentBikesOutDF = PreviousBikesOutDF[PreviousBikesOutDF['BikeID'].isin(JourneysToAdd['BikeID']) == False]

    #Limit trips that are added to Journey table to be in specific range of trip lenghts
    JourneysToAdd= JourneysToAdd[(JourneysToAdd['JourneyTime'] >= min_journey) & (JourneysToAdd['JourneyTime'] <= max_journey)]
    JourneysToAdd= JourneysToAdd[(JourneysToAdd['DateTimeOut'].dt.time >= end_time)]
    
    #Drop the Journey Time field as it is not require for DB table as we have start and return times in dataframe
    #Journey Time not really required or "normal form"
    JourneysToAdd.drop('JourneyTime', axis=1, inplace=True)   
    JourneysToAdd = JourneysToAdd.reset_index(drop=True) 


    #Bikes to Add to Bikes Out 
    BikesGoingOutDF = PreviousBikesLocationsDF[~is_duplicate_in_previous].copy()
    
    BikesGoingOutDF['DateTimeOut'] = None
    BikesGoingOutDF['DateTimeOut']=(pd.to_datetime(BikesGoingOutDF['DateTimeOut'])).dt.strftime("%m/%d/%Y, %H:%M:%S")
    BikesGoingOutDF['DateTimeOut']= times
    BikesGoingOutDF.reset_index(drop=True, inplace=True)
  

    #Append the new bikes going out to the Current Bikes Out DF
    CurrentBikesOutDF=CurrentBikesOutDF.append(BikesGoingOutDF, ignore_index = True)
    
    #Append the new completed journeys to journeys master list
    JourneysDF=JourneysDF.append(JourneysToAdd, ignore_index = True)

    
#Finally update all the CSV files. When all the historic data iterated through this will be uploaded to database tables
    
  
CurrentBikesLocationsDF.to_csv(basepath+'BikesLocations.csv', index= False)
CurrentBikesOutDF.to_csv(basepath+'BikesOut.csv', index= False)
JourneysDF.to_csv(basepath+'Journeys.csv', index= False)
